In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from alphabet_detector import AlphabetDetector
import unicodedata
from langdetect import detect
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords

nltk.download('stopwords')

cnchar_df = pd.read_excel('name_data/exigerData/EXGR_Chinese names(Character).xlsx')
cnrom_df = pd.read_excel("name_data/exigerData/EXGR_Chinese names(Romantized).xlsx")
viet_df = pd.read_excel("name_data/exigerData/EXGR_Vietnamese.xlsx")

pd.set_option('display.max_columns', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joliehuang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Todo:
- clean: chinese char, chinese rom, viet

In [2]:
#FUNCTIONS

#alphabet
def langname(name):
    if isinstance(name, str):
        result = []
        for char in name:
            try:
                char_name = unicodedata.name(char).split(' ')[0]
                result.append(char_name)
            except ValueError:
                # Handle characters without Unicode names
                result.append(char)
        return result
    else:
        return None  # Handle non-string or NaN values

#num_tokens (how many parts to the name?)
def calculate_token_length(name):
    if isinstance(name, str):
        # Check if the name contains Chinese characters
        if any('\u4e00' <= char <= '\u9fff' for char in name):
            # For Chinese names, count characters
            return len(name)
        else:
            # For non-Chinese names, split by spaces and count words
            return len(name.split())
    else:
        return None  # Handle non-string or NaN values

#char_ngrams
def generate_char_ngrams(text):
    if isinstance(text, str):
        # Tokenize the text into words
        words = text.split()
        
        # Create unigrams, bigrams, and trigrams
        unigrams = list(ngrams(text, 1))
        bigrams = list(ngrams(text, 2))
        trigrams = list(ngrams(text, 3))
        
        # Interpolated n-grams (combining unigrams, bigrams, and trigrams)
        interpolated_ngrams = unigrams + bigrams + trigrams
        
        return interpolated_ngrams
    else:
        return []

#name_length (length of the entire name string)
def get_name_length(fullname):
    if isinstance(fullname, str):
        return len(fullname)
    else:
        return np.nan 

#unigrams
def generate_char_unigrams(text):
    if isinstance(text, str):
        # Tokenize the text into characters
        characters = list(text)
        
        # Create unigrams
        unigrams = list(ngrams(characters, 1))
        
        return unigrams
    else:
        return []

#bigrams
def generate_char_bigrams(text):
    if isinstance(text, str):
        # Tokenize the text into characters
        characters = list(text)
        
        # Create unigrams
        bigrams = list(ngrams(characters, 2))
        
        return bigrams
    else:
        return []

#trigrams
def generate_char_trigrams(text):
    if isinstance(text, str):
        # Tokenize the text into characters
        characters = list(text)
        
        # Create unigrams
        trigrams = list(ngrams(characters, 3))
        
        return trigrams
    else:
        return []

EXGR_Chinese names(Character).xlsx - CLEANING
===

In [3]:
pd.set_option('display.max_columns', None)
# cnchar_df

In [4]:
duplicates = cnchar_df['fullname'].duplicated()
# print(cnchar_df[duplicates])

In [5]:
cnchar_df.drop_duplicates(subset=['fullname'], keep='first', inplace=True)
# cnchar_df

In [6]:
# null_values = cnchar_df['fullname'].isnull()
# print(cnchar_df[null_values])
# NO NULL VALUES

DATA EXPLORATION

In [7]:
# cnchar_df['Notes'].unique()

**Maybe notes saying that it's a minority name/non-native name should be counted? Drop all the ones that say pseudonym/stage name? Maybe we can afford to drop certain examples of names, since we have so many of them...?

In [8]:
drop_notes = ['minority name', 'non-native', 'pseudonym', 'stage name',
       'should be侯赛因.江；minority name',
       '劉-family name of her husband;吳-her family name', 'satage name',
       '周-family name of her husband;梁-her family name', 'fake name',
       'pen name', '方-family name of her husband; 黄-her family name',
       '曹-family name of her husband;王-her family name',
       '朱-family name of her husband;李-her family name',
       '朱-family name of her husband;葉-her family name',
       '林-family name of her husband;鄭-her family name',
       '梁-family name of her husband;劉- her family name ',
       '梁-family name of her husband;高- her family name',
       'should be 泰迪.羅賓；non-native',
       '罗-family name of her husband;范-her family name',
       '范-family name of her husband;徐-her family name',
       '葉-family name of her husband；劉-her family name',
       '蘇-family name of her husband;周- her family name', 'buddhist',
       'family name should be in Chinese character 周',
       '陈-family name of her husband; 冯-her family name',
       '陳-family name of her husband;方-her family name',
       '高-family name of her husband;金-her family name',
       '黃-family name of her husband;馬-her family name']

cnchar_df = cnchar_df[~cnchar_df['Notes'].isin(drop_notes)]
# cnchar_df

In [9]:
cnchar_df['Notes'].unique()

array([nan], dtype=object)

In [10]:
cnchar_df['Unnamed: 6'].unique()

array(['**Some commom compound Chinese surnames:东方，端木，独孤，公孙，上官，令狐，慕容，欧阳，司马，司徒，夏侯，西门，长孙，诸葛。',
       nan], dtype=object)

In [11]:
cnchar_df.drop(columns = ['Unnamed: 0', 'id', 'Family name', 'Given Name', 'Notes' ,'Unnamed: 6'], inplace = True)
# cnchar_df

Feature Engineering (Chinese Character)

In [12]:
cnchar_df['alphabet'] = cnchar_df['fullname'].apply(langname)
# cnchar_df

In [13]:
cnchar_df['word_length'] = cnchar_df['fullname'].apply(get_name_length)
# cnchar_df

In [14]:
cnchar_df['num_tokens'] = cnchar_df['fullname'].apply(calculate_token_length)
# cnchar_df

In [15]:
cnchar_df['char_ngrams'] = cnchar_df['fullname'].apply(generate_char_ngrams)
# cnchar_df

In [16]:
cnchar_df['unigrams'] = cnchar_df['fullname'].apply(generate_char_unigrams)
cnchar_df['bigrams'] = cnchar_df['fullname'].apply(generate_char_bigrams)
cnchar_df['trigrams'] = cnchar_df['fullname'].apply(generate_char_trigrams)
# cnchar_df

In [17]:
cnchar_df['period_freq'] = cnchar_df['fullname'].apply(lambda name: name.count('.'))
cnchar_df['dash_freq'] = cnchar_df['fullname'].apply(lambda name: name.count('-'))
cnchar_df['space_freq'] = cnchar_df['fullname'].apply(lambda name: name.count(' '))
cnchar_df['apostrophe_freq'] = cnchar_df['fullname'].apply(lambda name: name.count('\''))
cnchar_df

,fullname,alphabet,word_length,num_tokens,char_ngrams,unigrams,bigrams,trigrams,period_freq,dash_freq,space_freq,apostrophe_freq
0,丁一平,"[CJK, CJK, CJK]",3,3,"[(丁,), (一,), (平,), (丁, 一), (一, 平), (丁, 一, 平)]","[(丁,), (一,), (平,)]","[(丁, 一), (一, 平)]","[(丁, 一, 平)]",0,0,0,0
1,丁世雄,"[CJK, CJK, CJK]",3,3,"[(丁,), (世,), (雄,), (丁, 世), (世, 雄), (丁, 世, 雄)]","[(丁,), (世,), (雄,)]","[(丁, 世), (世, 雄)]","[(丁, 世, 雄)]",0,0,0,0
2,丁亦昕,"[CJK, CJK, CJK]",3,3,"[(丁,), (亦,), (昕,), (丁, 亦), (亦, 昕), (丁, 亦, 昕)]","[(丁,), (亦,), (昕,)]","[(丁, 亦), (亦, 昕)]","[(丁, 亦, 昕)]",0,0,0,0
3,丁仲礼,"[CJK, CJK, CJK]",3,3,"[(丁,), (仲,), (礼,), (丁, 仲), (仲, 礼), (丁, 仲, 礼)]","[(丁,), (仲,), (礼,)]","[(丁, 仲), (仲, 礼)]","[(丁, 仲, 礼)]",0,0,0,0
4,丁伟,"[CJK, CJK]",2,2,"[(丁,), (伟,), (丁, 伟)]","[(丁,), (伟,)]","[(丁, 伟)]",[],0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
12038,龚翔宇,"[CJK, CJK, CJK]",3,3,"[(龚,), (翔,), (宇,), (龚, 翔), (翔, 宇), (龚, 翔, 宇)]","[(龚,), (翔,), (宇,)]","[(龚, 翔), (翔, 宇)]","[(龚, 翔, 宇)]",0,0,0,0
12039,龚育之,"[CJK, CJK, CJK]",3,3,"[(龚,), (育,), (之,), (龚, 育), (育, 之), (龚, 育, 之)]","[(龚,), (育,), (之,)]","[(龚, 育), (育, 之)]","[(龚, 育, 之)]",0,0,0,0
12040,龚蓓苾,"[CJK, CJK, CJK]",3,3,"[(龚,), (蓓,), (苾,), (龚, 蓓), (蓓, 苾), (龚, 蓓, 苾)]","[(龚,), (蓓,), (苾,)]","[(龚, 蓓), (蓓, 苾)]","[(龚, 蓓, 苾)]",0,0,0,0
12041,龚贤永,"[CJK, CJK, CJK]",3,3,"[(龚,), (贤,), (永,), (龚, 贤), (贤, 永), (龚, 贤, 永)]","[(龚,), (贤,), (永,)]","[(龚, 贤), (贤, 永)]","[(龚, 贤, 永)]",0,0,0,0


EXGR_Chinese names(Romantized).xlsx - CLEANING
===

In [18]:
# cnrom_df

In [19]:
cnrom_df.drop(columns = ['Unnamed: 0', 'id'], inplace = True)

In [20]:
cnrom_df['Notes'].unique()

array(['non-native', nan], dtype=object)

In [21]:
duplicates = cnrom_df['fullname'].duplicated()
# print(cnrom_df[duplicates])

In [22]:
cnrom_df.drop_duplicates(subset=['fullname'], keep='first', inplace=True)
# cnrom_df

In [23]:
#Get rid of all examples with 'non-native', to try and further narrow df down to just Chinese names
cnrom_df.drop(cnrom_df[cnrom_df['Notes'] == 'non-native'].index, inplace=True)
# cnrom_df

In [24]:
cnrom_df['Notes'].unique()

array([nan], dtype=object)

In [25]:
cnrom_df['alphabet'] = cnrom_df['fullname'].apply(langname)
# cnrom_df

In [26]:
cnrom_df['word_length'] = cnrom_df['fullname'].apply(get_name_length)
# cnrom_df

In [27]:
cnrom_df['num_tokens'] = cnrom_df['fullname'].apply(calculate_token_length)
# cnrom_df

In [28]:
cnrom_df['char_ngrams'] = cnrom_df['fullname'].apply(generate_char_ngrams)
# cnrom_df

In [29]:
cnrom_df['unigrams'] = cnrom_df['fullname'].apply(generate_char_unigrams)
cnrom_df['bigrams'] = cnrom_df['fullname'].apply(generate_char_bigrams)
cnrom_df['trigrams'] = cnrom_df['fullname'].apply(generate_char_trigrams)
# cnrom_df

In [30]:
# cnrom_df['period_freq'] = cnrom_df['fullname'].apply(lambda name: name.count('.'))
# cnrom_df['dash_freq'] = cnrom_df['fullname'].apply(lambda name: name.count('-'))
# cnrom_df['space_freq'] = cnrom_df['fullname'].apply(lambda name: name.count(' '))
# cnrom_df['apostrophe_freq'] = cnrom_df['fullname'].apply(lambda name: name.count('\''))
cnrom_df

,fullname,Family name,Given Name,Notes,alphabet,word_length,num_tokens,char_ngrams,unigrams,bigrams,trigrams
2,Aaron Kwok,NaN,NaN,NaN,"[LATIN, LATIN, LATIN, LATIN, LATIN, SPACE, LAT...",10.0,2.0,"[(A,), (a,), (r,), (o,), (n,), ( ,), (K,), (w,...","[(A,), (a,), (r,), (o,), (n,), ( ,), (K,), (w,...","[(A, a), (a, r), (r, o), (o, n), (n, ), ( , K...","[(A, a, r), (a, r, o), (r, o, n), (o, n, ), (..."
9,Adhe Tapontsang,NaN,NaN,NaN,"[LATIN, LATIN, LATIN, LATIN, SPACE, LATIN, LAT...",15.0,2.0,"[(A,), (d,), (h,), (e,), ( ,), (T,), (a,), (p,...","[(A,), (d,), (h,), (e,), ( ,), (T,), (a,), (p,...","[(A, d), (d, h), (h, e), (e, ), ( , T), (T, a...","[(A, d, h), (d, h, e), (h, e, ), (e, , T), (..."
10,Ai Baojun,NaN,NaN,NaN,"[LATIN, LATIN, SPACE, LATIN, LATIN, LATIN, LAT...",9.0,2.0,"[(A,), (i,), ( ,), (B,), (a,), (o,), (j,), (u,...","[(A,), (i,), ( ,), (B,), (a,), (o,), (j,), (u,...","[(A, i), (i, ), ( , B), (B, a), (a, o), (o, j...","[(A, i, ), (i, , B), ( , B, a), (B, a, o), (..."
11,Ai Guoxiang,NaN,NaN,NaN,"[LATIN, LATIN, SPACE, LATIN, LATIN, LATIN, LAT...",11.0,2.0,"[(A,), (i,), ( ,), (G,), (u,), (o,), (x,), (i,...","[(A,), (i,), ( ,), (G,), (u,), (o,), (x,), (i,...","[(A, i), (i, ), ( , G), (G, u), (u, o), (o, x...","[(A, i, ), (i, , G), ( , G, u), (G, u, o), (..."
12,Ai Husheng,NaN,NaN,NaN,"[LATIN, LATIN, SPACE, LATIN, LATIN, LATIN, LAT...",10.0,2.0,"[(A,), (i,), ( ,), (H,), (u,), (s,), (h,), (e,...","[(A,), (i,), ( ,), (H,), (u,), (s,), (h,), (e,...","[(A, i), (i, ), ( , H), (H, u), (u, s), (s, h...","[(A, i, ), (i, , H), ( , H, u), (H, u, s), (..."
...,...,...,...,...,...,...,...,...,...,...,...
12009,Zu Xiaosun,NaN,NaN,NaN,"[LATIN, LATIN, SPACE, LATIN, LATIN, LATIN, LAT...",10.0,2.0,"[(Z,), (u,), ( ,), (X,), (i,), (a,), (o,), (s,...","[(Z,), (u,), ( ,), (X,), (i,), (a,), (o,), (s,...","[(Z, u), (u, ), ( , X), (X, i), (i, a), (a, o...","[(Z, u, ), (u, , X), ( , X, i), (X, i, a), (..."
12010,Zuo Shusheng,NaN,NaN,NaN,"[LATIN, LATIN, LATIN, SPACE, LATIN, LATIN, LAT...",12.0,2.0,"[(Z,), (u,), (o,), ( ,), (S,), (h,), (u,), (s,...","[(Z,), (u,), (o,), ( ,), (S,), (h,), (u,), (s,...","[(Z, u), (u, o), (o, ), ( , S), (S, h), (h, u...","[(Z, u, o), (u, o, ), (o, , S), ( , S, h), (..."
12011,Zuo Xiaoqing,NaN,NaN,NaN,"[LATIN, LATIN, LATIN, SPACE, LATIN, LATIN, LAT...",12.0,2.0,"[(Z,), (u,), (o,), ( ,), (X,), (i,), (a,), (o,...","[(Z,), (u,), (o,), ( ,), (X,), (i,), (a,), (o,...","[(Z, u), (u, o), (o, ), ( , X), (X, i), (i, a...","[(Z, u, o), (u, o, ), (o, , X), ( , X, i), (..."
12012,Zuo Yiteng,NaN,NaN,NaN,"[LATIN, LATIN, LATIN, SPACE, LATIN, LATIN, LAT...",10.0,2.0,"[(Z,), (u,), (o,), ( ,), (Y,), (i,), (t,), (e,...","[(Z,), (u,), (o,), ( ,), (Y,), (i,), (t,), (e,...","[(Z, u), (u, o), (o, ), ( , Y), (Y, i), (i, t...","[(Z, u, o), (u, o, ), (o, , Y), ( , Y, i), (..."


EXGR_Vietnamese.xlsx - CLEANING
===

In [31]:
# viet_df

In [32]:
viet_df.drop(columns = ['Unnamed: 0', 'fullname','Family name','Given name'], inplace = True)
viet_df = viet_df.rename(columns={'id': 'fullname'})

In [33]:
viet_df.drop_duplicates(subset=['fullname'], keep='first', inplace=True)
viet_df

,fullname
0,Từ Hoàng Thông
1,Nguyễn Thị Phương Thảo
2,Nguyễn Mạnh Dũng (cầu thủ bóng đá sinh 1981)
3,Nick Út
4,Cao Văn Lầu
...,...
3189,Nguyen Duc Kien
3190,Nguyen Phuc Thai
3192,Lê Phổ
3193,Vu Ngoc Nha


In [34]:
viet_df['alphabet'] = viet_df['fullname'].apply(langname)
# viet_df

In [35]:
viet_df['word_length'] = viet_df['fullname'].apply(get_name_length)
# viet_df

In [36]:
viet_df['num_tokens'] = viet_df['fullname'].apply(calculate_token_length)
# viet_df

In [37]:
viet_df['char_ngrams'] = viet_df['fullname'].apply(generate_char_ngrams)
# viet_df

In [38]:
viet_df['unigrams'] = viet_df['fullname'].apply(generate_char_unigrams)
viet_df['bigrams'] = viet_df['fullname'].apply(generate_char_bigrams)
viet_df['trigrams'] = viet_df['fullname'].apply(generate_char_trigrams)
# viet_df

In [39]:
viet_df['period_freq'] = viet_df['fullname'].apply(lambda name: name.count('.'))
viet_df['dash_freq'] = viet_df['fullname'].apply(lambda name: name.count('-'))
viet_df['space_freq'] = viet_df['fullname'].apply(lambda name: name.count(' '))
viet_df['apostrophe_freq'] = viet_df['fullname'].apply(lambda name: name.count('\''))

In [40]:
viet_df

,fullname,alphabet,word_length,num_tokens,char_ngrams,unigrams,bigrams,trigrams,period_freq,dash_freq,space_freq,apostrophe_freq
0,Từ Hoàng Thông,"[LATIN, LATIN, SPACE, LATIN, LATIN, LATIN, LAT...",14,3,"[(T,), (ừ,), ( ,), (H,), (o,), (à,), (n,), (g,...","[(T,), (ừ,), ( ,), (H,), (o,), (à,), (n,), (g,...","[(T, ừ), (ừ, ), ( , H), (H, o), (o, à), (à, n...","[(T, ừ, ), (ừ, , H), ( , H, o), (H, o, à), (...",0,0,2,0
1,Nguyễn Thị Phương Thảo,"[LATIN, LATIN, LATIN, LATIN, LATIN, LATIN, SPA...",22,4,"[(N,), (g,), (u,), (y,), (ễ,), (n,), ( ,), (T,...","[(N,), (g,), (u,), (y,), (ễ,), (n,), ( ,), (T,...","[(N, g), (g, u), (u, y), (y, ễ), (ễ, n), (n, ...","[(N, g, u), (g, u, y), (u, y, ễ), (y, ễ, n), (...",0,0,3,0
2,Nguyễn Mạnh Dũng (cầu thủ bóng đá sinh 1981),"[LATIN, LATIN, LATIN, LATIN, LATIN, LATIN, SPA...",44,9,"[(N,), (g,), (u,), (y,), (ễ,), (n,), ( ,), (M,...","[(N,), (g,), (u,), (y,), (ễ,), (n,), ( ,), (M,...","[(N, g), (g, u), (u, y), (y, ễ), (ễ, n), (n, ...","[(N, g, u), (g, u, y), (u, y, ễ), (y, ễ, n), (...",0,0,8,0
3,Nick Út,"[LATIN, LATIN, LATIN, LATIN, SPACE, LATIN, LATIN]",7,2,"[(N,), (i,), (c,), (k,), ( ,), (Ú,), (t,), (N,...","[(N,), (i,), (c,), (k,), ( ,), (Ú,), (t,)]","[(N, i), (i, c), (c, k), (k, ), ( , Ú), (Ú, t)]","[(N, i, c), (i, c, k), (c, k, ), (k, , Ú), (...",0,0,1,0
4,Cao Văn Lầu,"[LATIN, LATIN, LATIN, SPACE, LATIN, LATIN, LAT...",11,3,"[(C,), (a,), (o,), ( ,), (V,), (ă,), (n,), ( ,...","[(C,), (a,), (o,), ( ,), (V,), (ă,), (n,), ( ,...","[(C, a), (a, o), (o, ), ( , V), (V, ă), (ă, n...","[(C, a, o), (a, o, ), (o, , V), ( , V, ă), (...",0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3189,Nguyen Duc Kien,"[LATIN, LATIN, LATIN, LATIN, LATIN, LATIN, SPA...",15,3,"[(N,), (g,), (u,), (y,), (e,), (n,), ( ,), (D,...","[(N,), (g,), (u,), (y,), (e,), (n,), ( ,), (D,...","[(N, g), (g, u), (u, y), (y, e), (e, n), (n, ...","[(N, g, u), (g, u, y), (u, y, e), (y, e, n), (...",0,0,2,0
3190,Nguyen Phuc Thai,"[LATIN, LATIN, LATIN, LATIN, LATIN, LATIN, SPA...",16,3,"[(N,), (g,), (u,), (y,), (e,), (n,), ( ,), (P,...","[(N,), (g,), (u,), (y,), (e,), (n,), ( ,), (P,...","[(N, g), (g, u), (u, y), (y, e), (e, n), (n, ...","[(N, g, u), (g, u, y), (u, y, e), (y, e, n), (...",0,0,2,0
3192,Lê Phổ,"[LATIN, LATIN, SPACE, LATIN, LATIN, LATIN]",6,2,"[(L,), (ê,), ( ,), (P,), (h,), (ổ,), (L, ê), (...","[(L,), (ê,), ( ,), (P,), (h,), (ổ,)]","[(L, ê), (ê, ), ( , P), (P, h), (h, ổ)]","[(L, ê, ), (ê, , P), ( , P, h), (P, h, ổ)]",0,0,1,0
3193,Vu Ngoc Nha,"[LATIN, LATIN, SPACE, LATIN, LATIN, LATIN, LAT...",11,3,"[(V,), (u,), ( ,), (N,), (g,), (o,), (c,), ( ,...","[(V,), (u,), ( ,), (N,), (g,), (o,), (c,), ( ,...","[(V, u), (u, ), ( , N), (N, g), (g, o), (o, c...","[(V, u, ), (u, , N), ( , N, g), (N, g, o), (...",0,0,2,0
